In [41]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual

#sys.path.extend(['C:\\Users\\zj2hns\\wkspaces\\ASPE0000_00_Common\\aspe_package', 'C:/Users/zj2hns/wkspaces/ASPE0000_00_Common/aspe_package'])

from utils import calc_aspect_angle, calc_speed, calc_range, \
    filter_only_same_dir, filter_only_cars, filter_only_moving

In [33]:
dets_p = r"E:\logfiles\nexus_size_estimation_analysis\overtaking_detections.csv"
auto_gt_p = r"E:\logfiles\nexus_size_estimation_analysis\overtaking_auto_gt.csv"
dets = pd.read_csv(dets_p, index_col=0)
auto_gt = pd.read_csv(auto_gt_p, index_col=0)

In [34]:
print(dets.head(5))

   scan_index     timestamp  slot_id  sensor_id  position_x  position_y  \
0        9911  1.595825e+09      0.0          1   -4.491091   -6.360940   
1        9911  1.595825e+09      1.0          1   -4.904781   -5.960566   
2        9911  1.595825e+09      2.0          1   -4.332226   -7.448310   
3        9911  1.595825e+09      3.0          1   -5.663832   -6.033289   
4        9911  1.595825e+09      5.0          1   25.106241   -1.828816   

   azimuth_vcs  range_rate  range_rate_comp   even_name  event_id  \
0    -2.216606   -4.656250       -13.705444  overtaking         0   
1    -2.300187   -4.984375       -15.000218  overtaking         0   
2    -2.115005   -4.156250       -11.945992  overtaking         0   
3    -2.369966   -5.328125       -16.097702  overtaking         0   
4    -0.038127    8.421875        23.405277  overtaking         0   

   assoc_gt_unique_id  tcs_norm_pos_x  tcs_norm_pos_y  
0                54.0        0.909714        0.622026  
1                54.0 

In [37]:
sub_auto_gt_df = auto_gt.loc[:, ['event_id', 'scan_index', 'unique_id']]
sub_auto_gt_df.loc[:, 'auto_gt_df_index'] = sub_auto_gt_df.index.to_numpy()
sub_auto_gt_df.set_index(['event_id', 'scan_index', 'unique_id'], inplace=True)

dets_joined = dets.join(sub_auto_gt_df, on=['event_id', 'scan_index', 'assoc_gt_unique_id'])
auto_gt = filter_only_cars(auto_gt)
dets_joined = dets_joined.loc[dets_joined.auto_gt_df_index.isin(auto_gt.index), :]
print(dets_joined)

         scan_index     timestamp  slot_id  sensor_id  position_x  position_y  \
0              9911  1.595825e+09      0.0          1   -4.491091   -6.360940   
1              9911  1.595825e+09      1.0          1   -4.904781   -5.960566   
2              9911  1.595825e+09      2.0          1   -4.332226   -7.448310   
3              9911  1.595825e+09      3.0          1   -5.663832   -6.033289   
4              9911  1.595825e+09      5.0          1   25.106241   -1.828816   
...             ...           ...      ...        ...         ...         ...   
1234386       53497  1.595830e+09      7.0          1   29.531378   -2.785621   
1234387       53497  1.595830e+09      8.0          1   31.367790   -4.079490   
1234388       53497  1.595830e+09     37.0          2   29.466135   -1.900572   
1234389       53497  1.595830e+09     38.0          2   30.751677   -1.911349   
1234390       53497  1.595830e+09     39.0          2   32.256046   -3.844743   

         azimuth_vcs  range

In [39]:
@interact
def plot_heatmap(pos_x_center=(-80, 80, 1), 
                 pos_x_width=(1, 50, 1),
                 pos_y_center=(-10, 10, 1), 
                 pos_y_width=(1, 10, 1)):
    
    pos_x_min = pos_x_center - pos_x_width/2
    pos_x_max = pos_x_center + pos_x_width/2
    pos_y_min = pos_y_center - pos_y_width/2
    pos_y_max = pos_y_center + pos_y_width/2
    
    gt = auto_gt.query(f'({pos_x_min} < center_x) & (center_x < {pos_x_max}) &'
                   f'({pos_y_min} < center_y) & (center_y < {pos_y_max})')
    
    
    df = dets_joined.loc[dets_joined.auto_gt_df_index.isin(gt.index), :]
    
    #df = dets.query(f'({pos_x_min} < position_x) & (position_x < {pos_x_max}) &'
    #               f'({pos_y_min} < position_y) & (position_y < {pos_y_max})')
    
    auto_gt_df_indexes = df.loc[:, 'auto_gt_df_index'].unique()
    
    heat_map_x = df.loc[:, 'tcs_norm_pos_y'].to_numpy()
    heat_map_y = df.loc[:, 'tcs_norm_pos_x'].to_numpy()
    heatmap, xedges, yedges = np.histogram2d(heat_map_x, heat_map_y, bins=100, range=[[-2, 2], [-2, 2]])
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    
    fig = plt.figure(figsize=(15, 10))
    spec = fig.add_gridspec(2, 2)
    ax0 = fig.add_subplot(spec[:, 0])
    
    ax0.imshow(heatmap.T, extent=extent, origin='lower')
    ax0.set_xlim([-2, 2])
    ax0.set_ylim([-2, 2])
    
    ax1 = fig.add_subplot(spec[0, 1])
    df['tcs_norm_pos_x'].plot.hist(ax=ax1, bins=100, density=True)
    ax1.set_ylim([0, 3])
    ax1.set_xlim([-2.5, 2.5])
    ax1.grid()
    ax1.set_xlabel('tcs_norm_pos_x')
    
    ax2 = fig.add_subplot(spec[1, 1])
    df['tcs_norm_pos_y'].plot.hist(ax=ax2, bins=100, density=True)
    ax2.set_ylim([0, 3])
    ax2.set_xlim([-2.5, 2.5])

    ax2.grid()
    ax2.set_xlabel('tcs_norm_pos_y')


interactive(children=(IntSlider(value=0, description='pos_x_center', max=80, min=-80), IntSlider(value=25, des…